In [1]:
import datetime as dt
import pandas as pd
from wetterdienst.provider.dwd.observation import DwdObservationRequest
import numpy as np
import matplotlib.pyplot as plt
from pvlib import solarposition

In [2]:
parameters=[("hourly", "pressure"),("hourly", "air_temperature"),("hourly","moisture"),('10_minutes','solar'),('hourly','wind'),('hourly','precipitation')]

In [4]:
request = DwdObservationRequest(
    parameters=parameters,
    start_date=dt.datetime(2019, 1, 1),
    end_date=dt.datetime(2024, 1, 1)
)
# This downloads the data for station with id 00282 (Hassfurt)
stations = request.filter_by_station_id(station_id="00282")
df = stations.df
# From here you can query data by station
for result in stations.values.query():
    # analyse the station here
    break

df = pd.DataFrame(result.df.drop_nulls())
df.drop(columns=[0,6],inplace=True)
df.columns=['resolution','dataset','parameter','date','value']

In [5]:
df_solar=df.loc[df.resolution=='10_minutes']
df_other=df.loc[df.resolution=='hourly']

In [ ]:
# Transform the dataframe into a timeseries with distinct parameters as columns
# Use pivot_table to handle duplicate entries in the date column
df_solar_timeseries = df_solar.pivot_table(index='date', columns='parameter', values='value', aggfunc='mean')
df_other_timeseries = df_other.pivot_table(index='date', columns='parameter', values='value', aggfunc='mean')
# Reset the index to make it easier to work with
df_solar_timeseries.reset_index(inplace=True)
df_other_timeseries.reset_index(inplace=True)

df_solar_timeseries.index=pd.to_datetime(df_solar_timeseries.date)
df_solar_timeseries.drop(columns=['date'],inplace=True)
df_other_timeseries.index=pd.to_datetime(df_other_timeseries.date)
df_other_timeseries.drop(columns=['date'],inplace=True)

In [ ]:
# Create a new DataFrame for EPW data
df_other_timeseries_epw = df_other_timeseries[['temperature_air_mean_2m', 'temperature_dew_point_mean_2m', 'humidity', 'wind_direction', 'wind_speed']].copy()
df_other_timeseries_epw['pressure'] = df_other_timeseries['pressure_air_site'] * 100

In [ ]:
# Resample to hourly frequency, taking the mean for each hour
df_solar_timeseries = df_solar_timeseries.resample('h').sum()
# Convert radiation data from J/cm² to Wh/m²
conversion_factor = 2.7778
columns_to_convert = ['radiation_global', 'radiation_sky_short_wave_diffuse']  # Replace with your column names
df_solar_timeseries[columns_to_convert] = df_solar_timeseries[columns_to_convert] * conversion_factor

# Calculate solar position and DNI
latitude, longitude, altitude = 49.8743, 10.9206, 240
solpos = solarposition.get_solarposition(df_solar_timeseries.index, latitude, longitude, altitude=altitude)
df_solar_timeseries['zenith'] = solpos['apparent_zenith']
df_solar_timeseries['DNI'] = ((df_solar_timeseries['radiation_global'] - df_solar_timeseries['radiation_sky_short_wave_diffuse']) 
                               / np.cos(np.radians(df_solar_timeseries['zenith']))).clip(lower=0)

# Add DNI to EPW DataFrame
df_other_timeseries_epw['dni'] = df_solar_timeseries['DNI']
df_other_timeseries_epw['radiation_global'] = df_solar_timeseries['radiation_global']
df_other_timeseries_epw['radiation_diffuse'] = df_solar_timeseries['radiation_sky_short_wave_diffuse']
#floor the columns radiation_global, radiation_diffuse and dni at 0
columns_to_floor = ['radiation_global', 'radiation_diffuse', 'dni']
df_other_timeseries_epw[columns_to_floor] = df_other_timeseries_epw[columns_to_floor].clip(lower=0)

In [ ]:
df_other_timeseries_epw['year'] = df_other_timeseries_epw.index.year
df_other_timeseries_epw['month'] = df_other_timeseries_epw.index.month
df_other_timeseries_epw['day'] = df_other_timeseries_epw.index.day
df_other_timeseries_epw['hour'] = df_other_timeseries_epw.index.hour + 1  # EPW uses 1-24
df_other_timeseries_epw['minute'] = 0
# Example: fill other columns (replace with your actual data)
df_epw = pd.DataFrame({
    'year': df_other_timeseries_epw['year'].astype(int),
    'month': df_other_timeseries_epw['month'].astype(int),
    'day': df_other_timeseries_epw['day'].astype(int),
    'hour': df_other_timeseries_epw['hour'].astype(int),
    'minute': df_other_timeseries_epw['minute'].astype(int),
    'data source': "DWD",
    'dry_bulb': df_other_timeseries_epw['temperature_air_mean_2m'],
    'dew_point': df_other_timeseries_epw['temperature_dew_point_mean_2m'],
    'RH': df_other_timeseries_epw['humidity'],
    'pressure': df_other_timeseries_epw['pressure'],        # Pa
    'extraterrestrial horizontal radiation': 0,
    'extraterrestrial direct normal radiation': 0,
    'horizontal infrared Radiation Intensity': 0,
    'GHI': df_other_timeseries_epw['radiation_global'],
    'DNI': df_other_timeseries_epw['dni'],
    'DHI': df_other_timeseries_epw['radiation_diffuse'],
    'Global horizontal illuminence': 0,
    'Direct Normal Illuminence': 0,
    'diffuse horizontal illuminence':0,
    'zenith luminence':0,
    'wind_dir': df_other_timeseries_epw['wind_direction'],
    'wind_speed': df_other_timeseries_epw['wind_speed'],
    'sky_cover': 0,
    'opaque_sky_cover': 0,
    'visibility': 0,
    'ceiling_height': 0,
    'present_weather_observation': 0,
    'present_weather codes':0,
    'precip_water': 0,
    'aerosol_optical_depth': 0,
    'snow_depth': 0,
    'days since last snow': 99,
    'albedo':999,
    'liquid_precip_depth': 0,
    'liquid_precip_quantity': 0,
})

In [ ]:
# Define the complete range of time indices
full_index = pd.date_range(start=df_epw.index.min(), end=df_epw.index.max(), freq='h')

# Reindex the dataframe to include missing indices
df_epw = df_epw.reindex(full_index)

# Interpolate missing values using forward fill
df_epw.ffill(inplace=True)

C:\Users\jos82vs.UNI-WUERZBURG\AppData\Local\Temp\ipykernel_23436\1264406432.py:8: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_epw.ffill(inplace=True)


In [ ]:
for col in ['year', 'month', 'day', 'hour', 'minute']:
    df_epw[col] = df_epw[col].astype(int)

In [ ]:
def write_epw(df_epw, year):
    # --- 7. Write EPW header ---
    header_lines = [
        "LOCATION,Hassfurt.AB,BY,DEU,SRC-TMYx,107520,{:.4f},{:.4f},1.0,{:.1f}".format(latitude, longitude, altitude),
        "DESIGN CONDITIONS,0",
        "TYPICAL/EXTREME PERIODS,0",
        "GROUND TEMPERATURES,0",
        "HOLIDAYS/DAYLIGHT SAVINGS,No,0,0,0",
        "COMMENTS 1,Generated from DWD",
        "COMMENTS 2,Using Python",
        "DATA PERIODS,1,1,Data,Data,1/1,12/31"
    ]

    # --- 8. Write EPW file ---
    with open(f"haßfurt_weather_{year}.epw", "w", encoding="utf-8") as f:
        for line in header_lines:
            f.write(line + "\n")
        df_epw.to_csv(f, index=False, header=False, float_format="%.2f")

In [14]:
for year in range(2019, 2024):
    df_epw_year = df_epw[df_epw['year'] == year]
    write_epw(df_epw_year, year)


In [1]:
def epw_to_dataframe(weather_path):
    epw_labels = ['year', 'month', 'day', 'hour', 'minute', 'datasource', 'drybulb_C', 'dewpoint_C', 'relhum_percent',
                  'atmos_Pa', 'exthorrad_Whm2', 'extdirrad_Whm2', 'horirsky_Whm2', 'glohorrad_Whm2',
                  'dirnorrad_Whm2', 'difhorrad_Whm2', 'glohorillum_lux', 'dirnorillum_lux', 'difhorillum_lux',
                  'zenlum_lux', 'winddir_deg', 'windspd_ms', 'totskycvr_tenths', 'opaqskycvr_tenths', 'visibility_km',
                  'ceiling_hgt_m', 'presweathobs', 'presweathcodes', 'precip_wtr_mm', 'aerosol_opt_thousandths',
                  'snowdepth_cm', 'days_last_snow', 'Albedo', 'liq_precip_depth_mm', 'liq_precip_rate_Hour']

    df = pd.read_csv(weather_path, skiprows=8, header=None, names=epw_labels, encoding="utf-8").drop('datasource',
                                                                                                     axis=1)

    return df

In [3]:
import pandas as pd

In [6]:
weather_data=epw_to_dataframe('WetterDaten\haßfurt_weather_2023.epw')

In [13]:
# Check for problematic values in relhum_percent
problematic_relhum = weather_data[pd.to_numeric(weather_data['relhum_percent'], errors='coerce').isna()]
print("Problematic values in relhum_percent:")
print(problematic_relhum[['relhum_percent']])

weather_data['drybulb_C'].astype(float)

Problematic values in relhum_percent:
Empty DataFrame
Columns: [relhum_percent]
Index: []


0       13.0
1       12.5
2       12.4
3        7.4
4        5.4
        ... 
8755     6.3
8756     6.7
8757     6.2
8758     6.1
8759     5.5
Name: drybulb_C, Length: 8760, dtype: float64

In [ ]:
import math
import numpy as np
def epw_location(weather_path):
    """
    Returns the location of the EPW file as a tuple (latitude, longitude, utc, elevation).

    :param weather_path: Path to the EPW file
    :return: dict containing latitude, longitude, utc, and elevation
    """
    with open(weather_path, 'r') as f:
        epw_data = f.readlines()
        
    lat = float(epw_data[0].split(',')[6])
    lon = float(epw_data[0].split(',')[7])
    utc = float(epw_data[0].split(',')[8])
    elevation = float(epw_data[0].split(',')[9].strip("\n"))

    return {"latitude": lat, "longitude": lon, "utc": utc, "elevation": elevation}



def calc_wetbulb(Tdrybulb, RH):
    Tw = Tdrybulb * math.atan(0.151977 * ((RH + 8.313659) ** (0.5))) + math.atan(Tdrybulb + RH) - math.atan(
        RH - 1.676331) + (0.00391838 * (RH ** (3 / 2))) * math.atan(0.023101 * RH) - 4.686035

    return Tw  # wetbulb temperature in C

In [41]:
def epw_reader(weather_path):
    epw_data = epw_to_dataframe(weather_path)

    year = epw_data["year"][0]
    # Create date range from epw data
    date_range = pd.DatetimeIndex(
        pd.to_datetime(dict(year=epw_data.year, month=epw_data.month, day=epw_data.day, hour=epw_data.hour - 1))
    )
    epw_data['date'] = date_range
    epw_data['dayofyear'] = date_range.dayofyear

    try:
        epw_data['relhum_percent']=epw_data['relhum_percent'].astype(float)
        epw_data['drybulb_C']=epw_data['drybulb_C'].astype(float)
        epw_data['ratio_diffhout'] = epw_data['difhorrad_Whm2'] / epw_data['glohorrad_Whm2']
        epw_data['ratio_diffhout'] = epw_data['ratio_diffhout'].replace(np.inf, np.nan)
        epw_data['drybulb_C'] = pd.to_numeric(epw_data['drybulb_C'], errors='coerce')
        epw_data['relhum_percent'] = pd.to_numeric(epw_data['relhum_percent'], errors='coerce')
        epw_data['wetbulb_C'] = np.vectorize(calc_wetbulb)(epw_data['drybulb_C'], epw_data['relhum_percent'])
          
    except ValueError as e:
        raise ValueError(f"Errors found in the provided weather file: {e}") from e

    return epw_data

In [42]:
epw_data = epw_to_dataframe("WetterDaten/haßfurt_weather_2023.epw")
epw_data[epw_data==None].sum()

year                       0.0
month                      0.0
day                        0.0
hour                       0.0
minute                     0.0
drybulb_C                  0.0
dewpoint_C                 0.0
relhum_percent             0.0
atmos_Pa                   0.0
exthorrad_Whm2             0.0
extdirrad_Whm2             0.0
horirsky_Whm2              0.0
glohorrad_Whm2             0.0
dirnorrad_Whm2             0.0
difhorrad_Whm2             0.0
glohorillum_lux            0.0
dirnorillum_lux            0.0
difhorillum_lux            0.0
zenlum_lux                 0.0
winddir_deg                0.0
windspd_ms                 0.0
totskycvr_tenths           0.0
opaqskycvr_tenths          0.0
visibility_km              0.0
ceiling_hgt_m              0.0
presweathobs               0.0
presweathcodes             0.0
precip_wtr_mm              0.0
aerosol_opt_thousandths    0.0
snowdepth_cm               0.0
days_last_snow             0.0
Albedo                     0.0
liq_prec

In [43]:
epw_reader("WetterDaten/haßfurt_weather_2023.epw")

TypeError: float() argument must be a string or a real number, not 'complex'